# Qubit operator mapping

In this tutorial, we explain how to perform mapping from fermionic operators to qubit operators with QURI Parts, where we provide 3 types of mapping

1. Jordan-Wigner mapping
2. Bravyi-Kitaev mapping
3. Symmetry conserving Bravyi-Kitaev mapping


## Prerequisite

QURI Parts modules used in this tutorial: `quri-parts-chem` and `quri-parts-pyscf`. You can install them as follows:

In [1]:
#!pip install "quri_parts[chem]"
#!pip install "quri_parts[pyscf]"
#!pip install "quri_parts[openfermion]"

## Getting the Electron integrals and the Hamiltonian

We first start from a quick review of getting the electron integrals

In [2]:
# Step 1: Construct a PySCF Mole object
from pyscf import gto, scf
from quri_parts.pyscf.mol import PySCFMolecularOrbitals

h2_atom_list = [['H', [0, 0, 0]], ['H',  [0, 0, 1]]]
h2_mol = gto.Mole(atom=h2_atom_list, verbose = 0)
h2_mol.build()
h2_mf = scf.RHF(h2_mol)
h2_mf.kernel()
h2_mo = PySCFMolecularOrbitals(mol=h2_mol, mo_coeff=h2_mf.mo_coeff)

# Step 2: Compute the ao electron integrals without storing the electron integral arrays on memory
from quri_parts.pyscf.mol import get_ao_eint_set

ao_eint_set = get_ao_eint_set(h2_mo)

spin_mo_eint_set = ao_eint_set.to_full_space_mo_int(h2_mo)

# get the actual values and arrays of the electron integrals for contructing the hamiltonian
nuc_energy = spin_mo_eint_set.const
spin_mo_1e_int = spin_mo_eint_set.mo_1e_int.array.round(5)
spin_mo_2e_int = spin_mo_eint_set.mo_2e_int.array.round(5)

The moleculae Hamiltonian is given by 

\begin{equation}
H = E_{\text{nuc}} + h_{ij} c_i^{\dagger} c_j + \frac{1}{2} h_{ijkl} c_i^{\dagger} c_j^{\dagger} c_k c_l, \nonumber
\end{equation}

where $c_{i}$, $c_{i}^{\dagger}$ are the fermionic ladder operators and 

- $E_{\text{nuc}}$:     `nuc_energy`
- $h_{ij}$:     `spin_mo_1e_int`
- $h_{ijkl}$:   `spin_mo_2e_int`

After obtaining the electron integrals, we may use OpenFermion's `InteractionOperator` to convert the electron integrals to fermionic hamiltonian.
<a id='hamiltonian'></a>

In [3]:
from openfermion.ops import InteractionOperator

hamiltonian = InteractionOperator(
    constant= nuc_energy,
    one_body_tensor= spin_mo_1e_int,
    two_body_tensor= spin_mo_2e_int/2,  # Note that the factor of 1/2 is for consistent convention
)

## Convert Openfermion Operator to Quri-Parts Operator

We may convert any `openfermion` `QubitOperator`s into `quri-parts` `Operator` using the `operator_from_openfermion_op` functions

In [4]:
from openfermion.ops import QubitOperator
from quri_parts.openfermion.operator import operator_from_openfermion_op

In [5]:
of_op = 0.3 * QubitOperator('Y0 Z1 X2 Y3 X6') + 10 * QubitOperator('')
print(operator_from_openfermion_op(of_op))

0.3*Y0 Z1 X2 Y3 X6 + 10.0*I


For a given Hamiltonian, for example the one we just obtained [here](#hamiltonian), we may convert it to openfermion's `QubitOperator` with openfermion.jordan_wigner

and further convert it to quri-parts `Operator` with the `operator_from_openfermion_op` function.

In [6]:
from openfermion import jordan_wigner as of_jordan_wigner

for op, coeff in operator_from_openfermion_op(of_jordan_wigner(hamiltonian)).items():
    print(coeff.round(10), op)

(-0.3276002891+0j) I
(0.1371625+0j) Z0
(0.1371625+0j) Z1
(-0.130365+0j) Z2
(-0.130365+0j) Z3
(0.1566+0j) Z0 Z1
(0.10623+0j) Z0 Z2
(0.1554275+0j) Z0 Z3
(0.1554275+0j) Z1 Z2
(0.10623+0j) Z1 Z3
(0.1632675+0j) Z2 Z3
-0.0491975 X0 X1 Y2 Y3
0.0491975 X0 Y1 Y2 X3
0.0491975 Y0 X1 X2 Y3
-0.0491975 Y0 Y1 X2 X3


# Jordan-Wigner Mapping

`quri-parts` also provides Jordan Wigner mapping that can convert openfermion operators to quri-parts `Operator`.

It can also convert fermionic states to qubit states.

In [7]:
from quri_parts.openfermion.transforms import jordan_wigner

### Operator mapping

To perform Jordan-Wigner mapping, we first construct an operator mapper with the `jordan_wigner` object.

The `jordan_wigner.get_of_operator_mapper` object takes in the number of active space electrons and number of active space spin orbitals, 

then returns a function that maps

- `openfermion.ops.FermionOperator`
- `openfermion.ops.InteractionOperator`
- `openfermion.ops.MajoranaOperator`

to quri-parts `Operator`.


Here, we use the hamiltonian we defined [above](#hamiltonian), which is an `InteractionOperator` object, to demostrate how to obtain Jordan-Wigner hamiltonian written in terms of quri-parts `Operators`.

In [8]:
# Obtaining the operator mapper
operator_mapper = jordan_wigner.get_of_operator_mapper(
    n_fermions=h2_mo.n_electron,
    n_spin_orbitals=2*h2_mo.n_spatial_orb
)

# Map the Hamiltonian into quri-parts Operators with Jordan-Wigner mapping
jordan_wigner_hamiltonian = operator_mapper(hamiltonian)

for op, coeff in jordan_wigner_hamiltonian.items():
    print(coeff.round(10), op)

(-0.3276002891+0j) I
(0.1371625+0j) Z0
(0.1371625+0j) Z1
(-0.130365+0j) Z2
(-0.130365+0j) Z3
(0.1566+0j) Z0 Z1
(0.10623+0j) Z0 Z2
(0.1554275+0j) Z0 Z3
(0.1554275+0j) Z1 Z2
(0.10623+0j) Z1 Z3
(0.1632675+0j) Z2 Z3
-0.0491975 X0 X1 Y2 Y3
0.0491975 X0 Y1 Y2 X3
0.0491975 Y0 X1 X2 Y3
-0.0491975 Y0 Y1 X2 X3


### State Mapping

The state vector describing the occupation number can also be mapped to computational basis state using the `jordan_wigner` obect.

In this case, we may construct a state mapper function from the `jordan_wigner` object with the `get_state_mapper` method.

In [9]:
state_mapper = jordan_wigner.get_state_mapper(
    n_fermions=h2_mo.n_electron,
    n_spin_orbitals=2*h2_mo.n_spatial_orb
)

With the state mapper at hand, we may pass in the labels of the occupied spin orbitals to obtain a  Jordan-Wigner-mapped `ComputationalBasisState` object. 

For example, to generate the qubit state for
$$|\Psi\rangle = c_0^{\dagger}c_1^{\dagger}c_2^{\dagger} | 0\cdots 0 \rangle,$$
where the indices on the ladder operators are the spin-orbital indices, we can do:

In [10]:
occupation_spin_orbitals = [0, 1, 2]
jw_state = state_mapper([0, 1, 2])
jw_state_preparation_circuit = jw_state.circuit  # The circuit that prepares the specified state

print('State:')
print(jw_state, '\n')
print('State preparation circuit')
for gate in jw_state_preparation_circuit.gates:
    print(gate)

State:
ComputationalBasisState(qubit_count=4, bits=0b111, phase=0π/2) 

State preparation circuit
QuantumGate(name='X', target_indices=(0,), control_indices=(), params=(), pauli_ids=(), unitary_matrix=())
QuantumGate(name='X', target_indices=(1,), control_indices=(), params=(), pauli_ids=(), unitary_matrix=())
QuantumGate(name='X', target_indices=(2,), control_indices=(), params=(), pauli_ids=(), unitary_matrix=())


## Bravyi Kitaev mapping

`quri-parts` also provides Bravyi Kitaev mapping, the interface is the same as the `jordan_wigner`'s.

In [11]:
from quri_parts.openfermion.transforms import bravyi_kitaev

In [12]:
# Obtaining the operator mapper
bk_operator_mapper = bravyi_kitaev.get_of_operator_mapper(
    n_fermions=h2_mo.n_electron,
    n_spin_orbitals=2*h2_mo.n_spatial_orb
)

# Map the Hamiltonian into quri-parts Operators with Jordan-Wigner mapping
bravyi_kitaev_hamiltonian = bk_operator_mapper(hamiltonian)

for op, coeff in bravyi_kitaev_hamiltonian.items():
    print(coeff.round(10), op)

(0.1371625+0j) Z0
(-0.3276002891+0j) I
(0.1371625+0j) Z0 Z1
(-0.130365+0j) Z2
(-0.130365+0j) Z1 Z2 Z3
(0.0491975+0j) Y0 Z1 Y2
(0.0491975+0j) X0 Z1 X2
(0.0491975+0j) X0 Z1 X2 Z3
(0.0491975+0j) Y0 Z1 Y2 Z3
(0.1566+0j) Z1
(0.10623+0j) Z0 Z2
(0.1554275+0j) Z0 Z1 Z2
(0.1554275+0j) Z0 Z1 Z2 Z3
(0.10623+0j) Z0 Z2 Z3
(0.1632675+0j) Z1 Z3


In [13]:
bk_state_mapper = bravyi_kitaev.get_state_mapper(
    n_fermions=h2_mo.n_electron,
    n_spin_orbitals=2*h2_mo.n_spatial_orb
)

occupation_spin_orbitals = [0, 1, 2]
bk_state_mapper_state = state_mapper(occupation_spin_orbitals)
bk_state_preparation_circuit = bk_state_mapper_state.circuit

print('State:')
print(bk_state_mapper_state, '\n')
print('State preparation circuit:')
for gate in bk_state_preparation_circuit.gates:
    print(gate)

State:
ComputationalBasisState(qubit_count=4, bits=0b111, phase=0π/2) 

State preparation circuit:
QuantumGate(name='X', target_indices=(0,), control_indices=(), params=(), pauli_ids=(), unitary_matrix=())
QuantumGate(name='X', target_indices=(1,), control_indices=(), params=(), pauli_ids=(), unitary_matrix=())
QuantumGate(name='X', target_indices=(2,), control_indices=(), params=(), pauli_ids=(), unitary_matrix=())


## Symmetry Conserving Bravyi-Kitaev mapping

`quri-parts` also provides Symmetry Conserving Bravyi Kitaev mapping, the interface is the same as the `jordan_wigner`'s.

In [14]:
from quri_parts.openfermion.transforms import symmetry_conserving_bravyi_kitaev
from openfermion.transforms import get_fermion_operator

In [15]:
import numpy as np
# Obtaining the operator mapper
sym_conserving_bk_operator_mapper = symmetry_conserving_bravyi_kitaev.get_of_operator_mapper(
    n_fermions=h2_mo.n_electron,
    n_spin_orbitals=int(2 * h2_mo.n_spatial_orb)
)

# Map the Hamiltonian into quri-parts Operators with Jordan-Wigner mapping
sym_conserving_bravyi_kitaev_hamiltonian = sym_conserving_bk_operator_mapper(hamiltonian)

for op, coeff in sym_conserving_bravyi_kitaev_hamiltonian.items():
    if (c := np.round(coeff, 10)):
        print(c, op)

-0.5400602891 I
0.2675275 Z0
0.2675275 Z1
0.0090125 Z0 Z1
0.19679 X0 X1


In [16]:
scbk_state_mapper = symmetry_conserving_bravyi_kitaev.get_state_mapper(
    n_fermions=h2_mo.n_electron,
    n_spin_orbitals=int(2*h2_mo.n_spatial_orb)
)

occupation_spin_orbitals = [0, 1, 2]
scbk_state_mapper_state = state_mapper(occupation_spin_orbitals)
scbk_state_preparation_circuit = scbk_state_mapper_state.circuit

print('State:')
print(scbk_state_mapper_state, '\n')
print('State preparation circuit:')
for gate in scbk_state_preparation_circuit.gates:
    print(gate)

State:
ComputationalBasisState(qubit_count=4, bits=0b111, phase=0π/2) 

State preparation circuit:
QuantumGate(name='X', target_indices=(0,), control_indices=(), params=(), pauli_ids=(), unitary_matrix=())
QuantumGate(name='X', target_indices=(1,), control_indices=(), params=(), pauli_ids=(), unitary_matrix=())
QuantumGate(name='X', target_indices=(2,), control_indices=(), params=(), pauli_ids=(), unitary_matrix=())
